In [384]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.linear_model import LogisticRegression

import sys
sys.path.insert(1, '/Users/bolin/Desktop/NUS_Y4S2/BT4222/BT4222_repo')
from custom_utils import *

In [385]:
df = pd.read_csv("/Users/bolin/Desktop/NUS_Y4S2/BT4222/BT4222_repo/data/cooked_data/cooked_complete_dataset.csv")
# prevent 0 division zero
df = df.replace(0, 1)

In [386]:
df

,date,Adj_Close_BTC-USD,Open_BTC-USD,High_BTC-USD,Low_BTC-USD,Volume_BTC-USD,Adj_Close_SPY,Adj_Close_GLD,Adj_Close_CHFUSD=X,Adj_Close_CNYUSD=X,Adj_Close_EURUSD=X,Adj_Close_GBPUSD=X,Adj_Close_JPYUSD=X,coindesk_sentiment,num_of_coindesk_posts,reddit_comments_sentiments,top_50_reddit_posts_sentiments,blockchain_transactions_per_block,blockchain_hash_rates
0,2020-12-14,19246.64453,19144.49219,19305.09961,19012.70898,2.247400e+10,361.926788,171.539993,1.125442,0.152772,1.213340,1.331824,0.009621,0.249489,12,0.188275,0.297238,2167.931034,134533587.6
1,2020-12-15,19417.07617,19246.91992,19525.00781,19079.84180,2.674198e+10,366.819824,173.940002,1.127930,0.152679,1.214890,1.333084,0.009614,0.173773,18,0.144389,0.399427,2288.857143,133351912.2
2,2020-12-16,21310.59766,19418.81836,21458.90820,19298.31641,4.440901e+10,367.395508,174.899994,1.129382,0.152945,1.215430,1.344447,0.009649,0.341491,11,0.137256,0.489673,2204.314685,132323572.3
3,2020-12-17,22805.16211,21308.35156,23642.66016,21234.67578,7.137861e+10,369.449982,176.740005,1.129446,0.153109,1.219959,1.350293,0.009664,0.197572,10,0.156723,0.636030,2399.077519,132373208.7
4,2020-12-18,23137.96094,22806.79688,23238.60156,22399.81250,4.038790e+10,367.974792,176.440002,1.130301,0.153090,1.226272,1.357018,0.009696,0.315601,2,0.166419,0.107093,2392.031847,131791042.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,2021-04-02,59384.31250,59098.87890,60267.18750,58869.28130,5.870000e+10,400.609985,161.979996,1.061447,0.152337,1.177995,1.383509,0.009041,0.121955,10,0.146423,0.152625,2184.200000,163507250.5
110,2021-04-03,57603.89060,59397.41020,60110.26950,57603.89060,5.960000e+10,400.609985,161.979996,1.061447,0.152337,1.177995,1.383509,0.009041,0.501192,1,0.132042,0.274113,2150.511450,165320614.4
111,2021-04-04,58758.55470,57604.83980,58913.74610,57168.67580,5.080000e+10,400.609985,161.979996,1.061447,0.152337,1.177995,1.383509,0.009041,0.206128,1,0.216657,0.148517,1684.979452,162156064.8
112,2021-04-05,59057.87890,58760.87500,59891.29690,57694.82420,6.070000e+10,406.359985,161.919998,1.061571,0.152288,1.176817,1.383126,0.009035,0.179058,21,0.232894,0.351100,1852.858974,161456161.7


In [387]:
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

In [388]:
df.columns

Index(['date', 'Adj_Close_BTC-USD', 'Open_BTC-USD', 'High_BTC-USD',
       'Low_BTC-USD', 'Volume_BTC-USD', 'Adj_Close_SPY', 'Adj_Close_GLD',
       'Adj_Close_CHFUSD=X', 'Adj_Close_CNYUSD=X', 'Adj_Close_EURUSD=X',
       'Adj_Close_GBPUSD=X', 'Adj_Close_JPYUSD=X', 'coindesk_sentiment',
       'num_of_coindesk_posts', 'reddit_comments_sentiments',
       'top_50_reddit_posts_sentiments', 'blockchain_transactions_per_block',
       'blockchain_hash_rates'],
      dtype='object')

In [389]:
# conduct log return transformation on all columns
df = log_return_transformation(df)

In [390]:
# custom number of lags for selected columns
# columns that are not specify in this dictionary will be lagged by on by default
lags = {
    'Adj_Close_BTC-USD': 6,
    'Open_BTC-USD': 6,
    'Low_BTC-USD': 6,
    'Volume_BTC-USD': 6,
    'Adj_Close_SPY': 6,
    'Adj_Close_GLD': 6,
}

In [391]:
df = get_num_lags(df, lags)

/Users/bolin/Desktop/NUS_Y4S2/BT4222/BT4222_repo/custom_utils/get_num_lags.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column + '_lag_' + str(i)] = df[column].shift(i)


In [392]:
df.columns

Index(['date', 'Adj_Close_BTC-USD', 'Open_BTC-USD_lag_1', 'Open_BTC-USD_lag_2',
       'Open_BTC-USD_lag_3', 'Open_BTC-USD_lag_4', 'Open_BTC-USD_lag_5',
       'High_BTC-USD_lag_1', 'Low_BTC-USD_lag_1', 'Low_BTC-USD_lag_2',
       'Low_BTC-USD_lag_3', 'Low_BTC-USD_lag_4', 'Low_BTC-USD_lag_5',
       'Volume_BTC-USD_lag_1', 'Volume_BTC-USD_lag_2', 'Volume_BTC-USD_lag_3',
       'Volume_BTC-USD_lag_4', 'Volume_BTC-USD_lag_5', 'Adj_Close_SPY_lag_1',
       'Adj_Close_SPY_lag_2', 'Adj_Close_SPY_lag_3', 'Adj_Close_SPY_lag_4',
       'Adj_Close_SPY_lag_5', 'Adj_Close_GLD_lag_1', 'Adj_Close_GLD_lag_2',
       'Adj_Close_GLD_lag_3', 'Adj_Close_GLD_lag_4', 'Adj_Close_GLD_lag_5',
       'Adj_Close_CHFUSD=X_lag_1', 'Adj_Close_CNYUSD=X_lag_1',
       'Adj_Close_EURUSD=X_lag_1', 'Adj_Close_GBPUSD=X_lag_1',
       'Adj_Close_JPYUSD=X_lag_1', 'coindesk_sentiment_lag_1',
       'num_of_coindesk_posts_lag_1', 'reddit_comments_sentiments_lag_1',
       'top_50_reddit_posts_sentiments_lag_1',
       'blo

In [393]:
df['Adj_Close_BTC-USD'] = df['Adj_Close_BTC-USD'].apply(lambda x: 1 if x > 0 else 0)

In [394]:
train = df[df['date']<'2021-03-11'].sort_values('date')
test = df[df['date']>='2021-03-11'].sort_values('date')

In [395]:
train

,date,Adj_Close_BTC-USD,Open_BTC-USD_lag_1,Open_BTC-USD_lag_2,Open_BTC-USD_lag_3,Open_BTC-USD_lag_4,Open_BTC-USD_lag_5,High_BTC-USD_lag_1,Low_BTC-USD_lag_1,Low_BTC-USD_lag_2,...,Adj_Close_CNYUSD=X_lag_1,Adj_Close_EURUSD=X_lag_1,Adj_Close_GBPUSD=X_lag_1,Adj_Close_JPYUSD=X_lag_1,coindesk_sentiment_lag_1,num_of_coindesk_posts_lag_1,reddit_comments_sentiments_lag_1,top_50_reddit_posts_sentiments_lag_1,blockchain_transactions_per_block_lag_1,blockchain_hash_rates_lag_1
6,2020-12-20,0,0.014196,0.067960,0.092856,0.008892,0.005336,0.035810,0.018868,0.053417,...,0.000000,0.000000,0.000000,0.000000,-0.211596,1.252763,-0.566069,1.359533,-0.095446,0.015375
7,2020-12-21,0,0.031023,0.014196,0.067960,0.092856,0.008892,0.005127,0.013975,0.018868,...,0.000000,0.000000,0.000000,0.000000,0.068756,0.251314,0.444461,0.195889,-0.151917,0.022101
8,2020-12-22,1,-0.016365,0.031023,0.014196,0.067960,0.092856,-0.006202,-0.043636,0.013975,...,-0.000597,0.000000,-0.012474,-0.003156,-0.063549,0.000000,0.192906,0.061448,0.060372,-0.008725
9,2020-12-23,0,-0.029414,-0.016365,0.031023,0.014196,0.067960,-0.011289,0.012166,-0.043636,...,-0.001865,-0.002107,0.003544,0.001132,0.202284,0.441833,-0.478721,-0.012084,0.133250,-0.008837
10,2020-12-24,1,0.042429,-0.029414,-0.016365,0.031023,0.014196,0.009813,0.016450,0.012166,...,0.000917,-0.004115,-0.003732,-0.001875,-0.199405,-0.336472,0.339762,-0.408491,-0.030940,0.002954
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,2021-03-06,0,-0.040294,0.042588,-0.024408,0.094034,-0.022650,-0.046258,-0.023662,-0.012872,...,-0.000340,-0.006905,-0.002365,-0.008344,0.088277,-0.257829,-0.042894,0.336420,-0.059164,-0.028171
83,2021-03-07,1,0.007641,-0.040294,0.042588,-0.024408,0.094034,-0.005058,0.015246,-0.023662,...,0.000000,0.000000,0.000000,0.000000,0.107990,-1.734601,-0.144834,-0.403195,-0.118462,0.010152
84,2021-03-08,1,0.000398,0.007641,-0.040294,0.042588,-0.024408,0.044514,0.034547,0.015246,...,0.000000,0.000000,0.000000,0.000000,0.542306,-1.098612,0.121371,0.180079,-0.059038,-0.006844
85,2021-03-09,1,0.045075,0.000398,0.007641,-0.040294,0.042588,0.017931,0.011936,0.034547,...,-0.004119,-0.003524,-0.002913,-0.004289,-0.574569,2.944439,0.108409,0.058803,0.053261,0.013281


In [396]:
test

,date,Adj_Close_BTC-USD,Open_BTC-USD_lag_1,Open_BTC-USD_lag_2,Open_BTC-USD_lag_3,Open_BTC-USD_lag_4,Open_BTC-USD_lag_5,High_BTC-USD_lag_1,Low_BTC-USD_lag_1,Low_BTC-USD_lag_2,...,Adj_Close_CNYUSD=X_lag_1,Adj_Close_EURUSD=X_lag_1,Adj_Close_GBPUSD=X_lag_1,Adj_Close_JPYUSD=X_lag_1,coindesk_sentiment_lag_1,num_of_coindesk_posts_lag_1,reddit_comments_sentiments_lag_1,top_50_reddit_posts_sentiments_lag_1,blockchain_transactions_per_block_lag_1,blockchain_hash_rates_lag_1
87,2021-03-11,1,0.047649,0.021246,0.045075,0.000398,0.007641,0.043442,0.024871,0.048799,...,0.002916,0.003890,0.004822,0.003329,-0.126294,0.174353,-0.170926,0.081567,0.077614,-0.004698
88,2021-03-12,0,0.020566,0.047649,0.021246,0.045075,0.000398,0.014440,0.022153,0.024871,...,0.000108,0.002363,0.002949,0.001336,0.481252,-0.083382,-0.109839,-0.258875,-0.047330,-0.004833
89,2021-03-13,1,0.032662,0.020566,0.047649,0.021246,0.045075,-0.001627,0.016240,0.022153,...,0.001831,0.005022,0.004188,-0.001134,-0.408116,-0.044452,-0.360870,-0.190262,-0.031355,0.002299
90,2021-03-14,0,-0.008299,0.032662,0.020566,0.047649,0.021246,0.061638,0.015079,0.016240,...,0.000000,0.000000,0.000000,0.000000,1.722283,-3.091042,-0.036499,-0.304325,-0.098575,-0.000894
91,2021-03-15,0,0.065435,-0.008299,0.032662,0.020566,0.047649,-0.001394,0.053412,0.015079,...,0.000000,0.000000,0.000000,0.000000,-1.180230,0.693147,-0.720337,-0.076354,0.012584,-0.009004
92,2021-03-16,1,-0.032433,0.065435,-0.008299,0.032662,0.020566,-0.017307,-0.068192,0.053412,...,-0.000847,-0.003016,-0.004327,-0.005164,-1.297682,2.397895,0.929648,0.507123,0.006316,0.010779
93,2021-03-17,1,-0.059555,-0.032433,0.065435,-0.008299,0.032662,-0.063200,-0.033747,-0.068192,...,-0.000062,-0.002029,-0.002185,-0.000156,0.391725,-0.451985,0.286366,-0.215417,0.082457,-0.006846
94,2021-03-18,0,0.017486,-0.059555,-0.032433,0.065435,-0.008299,0.036905,0.018016,-0.033747,...,-0.000938,-0.002025,0.000222,0.000963,0.706084,0.356675,-0.432622,0.278119,0.015317,0.008793
95,2021-03-19,1,0.035733,0.017486,-0.059555,-0.032433,0.065435,0.019254,-0.005057,0.018016,...,0.000369,0.006795,0.005283,0.001267,0.065935,0.139762,0.372903,-0.131317,0.002298,-0.012727
96,2021-03-20,0,-0.017862,0.035733,0.017486,-0.059555,-0.032433,-0.010331,0.043112,-0.005057,...,-0.000338,-0.005402,-0.003641,-0.000367,-0.408116,-0.044452,-0.453221,0.202507,-0.021305,0.019513


In [397]:
y_train = train['Adj_Close_BTC-USD']
X_train = train.iloc[:,2:]
X_train_no_sentiment = X_train.drop(['coindesk_sentiment_lag_1', 'num_of_coindesk_posts_lag_1', 'reddit_comments_sentiments_lag_1', 'top_50_reddit_posts_sentiments_lag_1'], axis=1)


y_test = test['Adj_Close_BTC-USD']
X_test = test.iloc[:,2:]
X_test_no_sentiment = X_test.drop(['coindesk_sentiment_lag_1', 'num_of_coindesk_posts_lag_1', 'reddit_comments_sentiments_lag_1', 'top_50_reddit_posts_sentiments_lag_1'], axis=1)


In [398]:
y_train.value_counts()

1    46
0    35
Name: Adj_Close_BTC-USD, dtype: int64

In [399]:
y_test.value_counts()

1    15
0    11
Name: Adj_Close_BTC-USD, dtype: int64

### Logistic regression with complete set of independent variables (With sentiments)

In [400]:
lr = LogisticRegression(max_iter=500, solver='liblinear')
lr.fit(X_train, y_train)

LogisticRegression(max_iter=500, solver='liblinear')

In [401]:
y_pred_class = lr.predict(X_train)
print('Accuracy score in training set: {}'.format(metrics.accuracy_score(y_train, y_pred_class)))
print('Precision score in training set: {}'.format(metrics.precision_score(y_train, y_pred_class)))
print('Recall score in training set: {}'.format(metrics.recall_score(y_train, y_pred_class)))
print('Confusion matrix:')
print(metrics.confusion_matrix(y_train, y_pred_class))

y_pred_prob = lr.predict_proba(X_train)[:, 1]
print('ROC_AUC score in training set: {}'.format(metrics.roc_auc_score(y_train, y_pred_prob)))

print('*'*80)

y_pred_class = lr.predict(X_test)
print('Accuracy score in test set: {}'.format(metrics.accuracy_score(y_test, y_pred_class)))
print('Precision score in test set: {}'.format(metrics.precision_score(y_test, y_pred_class)))
print('Recall score in test set: {}'.format(metrics.recall_score(y_test, y_pred_class)))
print('Confusion matrix:')
print(metrics.confusion_matrix(y_test, y_pred_class))

y_pred_prob = lr.predict_proba(X_test)[:, 1]
print('ROC_AUC score in test set: {}'.format(metrics.roc_auc_score(y_test, y_pred_prob)))

Accuracy score in training set: 0.654320987654321
Precision score in training set: 0.65
Recall score in training set: 0.8478260869565217
Confusion matrix:
[[14 21]
 [ 7 39]]
ROC_AUC score in training set: 0.6819875776397516
********************************************************************************
Accuracy score in test set: 0.5384615384615384
Precision score in test set: 0.5789473684210527
Recall score in test set: 0.7333333333333333
Confusion matrix:
[[ 3  8]
 [ 4 11]]
ROC_AUC score in test set: 0.48484848484848486


### Logistic regression with complete set of independent variables (Without sentiments)

In [402]:
lr = LogisticRegression(max_iter=500, solver='liblinear')
lr.fit(X_train_no_sentiment, y_train)

LogisticRegression(max_iter=500, solver='liblinear')

In [403]:
y_pred_class = lr.predict(X_train_no_sentiment)
print('Accuracy score in training set: {}'.format(metrics.accuracy_score(y_train, y_pred_class)))
print('Precision score in training set: {}'.format(metrics.precision_score(y_train, y_pred_class)))
print('Recall score in training set: {}'.format(metrics.recall_score(y_train, y_pred_class)))
print('Confusion matrix:')
print(metrics.confusion_matrix(y_train, y_pred_class))

y_pred_prob = lr.predict_proba(X_train_no_sentiment)[:, 1]
print('ROC_AUC score in training set: {}'.format(metrics.roc_auc_score(y_train, y_pred_prob)))

print('*'*80)

y_pred_class = lr.predict(X_test_no_sentiment)
print('Accuracy score in test set: {}'.format(metrics.accuracy_score(y_test, y_pred_class)))
print('Precision score in test set: {}'.format(metrics.precision_score(y_test, y_pred_class)))
print('Recall score in test set: {}'.format(metrics.recall_score(y_test, y_pred_class)))
print('Confusion matrix:')
print(metrics.confusion_matrix(y_test, y_pred_class))

y_pred_prob = lr.predict_proba(X_test_no_sentiment)[:, 1]
print('ROC_AUC score in test set: {}'.format(metrics.roc_auc_score(y_test, y_pred_prob)))

Accuracy score in training set: 0.6172839506172839
Precision score in training set: 0.6119402985074627
Recall score in training set: 0.8913043478260869
Confusion matrix:
[[ 9 26]
 [ 5 41]]
ROC_AUC score in training set: 0.6447204968944099
********************************************************************************
Accuracy score in test set: 0.46153846153846156
Precision score in test set: 0.5217391304347826
Recall score in test set: 0.8
Confusion matrix:
[[ 0 11]
 [ 3 12]]
ROC_AUC score in test set: 0.32727272727272727


### Logistic regression with selected set of independent variables from stepwise regression (With Sentiment)

In [404]:
selected_x_variables = "coindesk_sentiment_lag_1 + Adj_Close_GLD_lag_2 + top_50_reddit_posts_sentiments_lag_1 + Volume_BTC-USD_lag_3 + blockchain_transactions_per_block_lag_1"
selected_x_variables = selected_x_variables.split(' + ')

In [405]:
selected_x_variables

['coindesk_sentiment_lag_1',
 'Adj_Close_GLD_lag_2',
 'top_50_reddit_posts_sentiments_lag_1',
 'Volume_BTC-USD_lag_3',
 'blockchain_transactions_per_block_lag_1']

In [406]:
new_X_train = X_train[selected_x_variables]
new_X_test = X_test[selected_x_variables]

In [407]:
lr = LogisticRegression(max_iter=500, solver='liblinear')
lr.fit(new_X_train, y_train)

LogisticRegression(max_iter=500, solver='liblinear')

In [408]:
output_with_sentiment = []
y_pred_class = lr.predict(new_X_train)
output_with_sentiment = output_with_sentiment + list(y_pred_class)

print('Accuracy score in training set: {}'.format(metrics.accuracy_score(y_train, y_pred_class)))
print('Precision score in training set: {}'.format(metrics.precision_score(y_train, y_pred_class)))
print('Recall score in training set: {}'.format(metrics.recall_score(y_train, y_pred_class)))
print('Confusion matrix:')
print(metrics.confusion_matrix(y_train, y_pred_class))

y_pred_prob = lr.predict_proba(new_X_train)[:, 1]
print('ROC_AUC score in training set: {}'.format(metrics.roc_auc_score(y_train, y_pred_prob)))

print('*'*80)

y_pred_class = lr.predict(new_X_test)
output_with_sentiment = output_with_sentiment + list(y_pred_class)

print('Accuracy score in test set: {}'.format(metrics.accuracy_score(y_test, y_pred_class)))
print('Precision score in test set: {}'.format(metrics.precision_score(y_test, y_pred_class)))
print('Recall score in test set: {}'.format(metrics.recall_score(y_test, y_pred_class)))
print('Confusion matrix:')
print(metrics.confusion_matrix(y_test, y_pred_class))

y_pred_prob = lr.predict_proba(new_X_test)[:, 1]
print('ROC_AUC score in test set: {}'.format(metrics.roc_auc_score(y_test, y_pred_prob)))

Accuracy score in training set: 0.5679012345679012
Precision score in training set: 0.5873015873015873
Recall score in training set: 0.8043478260869565
Confusion matrix:
[[ 9 26]
 [ 9 37]]
ROC_AUC score in training set: 0.6322981366459628
********************************************************************************
Accuracy score in test set: 0.6153846153846154
Precision score in test set: 0.631578947368421
Recall score in test set: 0.8
Confusion matrix:
[[ 4  7]
 [ 3 12]]
ROC_AUC score in test set: 0.6363636363636364


### Logistic regression with selected set of independent variables from stepwise regression (Without Sentiment) 

In [409]:
selected_x_variables = "Adj_Close_GLD_lag_1 + Adj_Close_GLD_lag_2 + Volume_BTC-USD_lag_5 + Volume_BTC-USD_lag_3"
selected_x_variables = selected_x_variables.split(' + ')

In [410]:
selected_x_variables

['Adj_Close_GLD_lag_1',
 'Adj_Close_GLD_lag_2',
 'Volume_BTC-USD_lag_5',
 'Volume_BTC-USD_lag_3']

In [411]:
new_X_train = X_train[selected_x_variables]
new_X_test = X_test[selected_x_variables]

In [412]:
lr = LogisticRegression(max_iter=500, solver='liblinear')
lr.fit(new_X_train, y_train)

LogisticRegression(max_iter=500, solver='liblinear')

In [413]:
output_without_sentiment = []
y_pred_class = lr.predict(new_X_train)
output_without_sentiment = output_without_sentiment + list(y_pred_class)
print('Accuracy score in training set: {}'.format(metrics.accuracy_score(y_train, y_pred_class)))
print('Precision score in training set: {}'.format(metrics.precision_score(y_train, y_pred_class)))
print('Recall score in training set: {}'.format(metrics.recall_score(y_train, y_pred_class)))
print('Confusion matrix:')
print(metrics.confusion_matrix(y_train, y_pred_class))

y_pred_prob = lr.predict_proba(new_X_train)[:, 1]
print('ROC_AUC score in training set: {}'.format(metrics.roc_auc_score(y_train, y_pred_prob)))

print('*'*80)

y_pred_class = lr.predict(new_X_test)
output_without_sentiment = output_without_sentiment + list(y_pred_class)
print('Accuracy score in test set: {}'.format(metrics.accuracy_score(y_test, y_pred_class)))
print('Precision score in test set: {}'.format(metrics.precision_score(y_test, y_pred_class)))
print('Recall score in test set: {}'.format(metrics.recall_score(y_test, y_pred_class)))
print('Confusion matrix:')
print(metrics.confusion_matrix(y_test, y_pred_class))

y_pred_prob = lr.predict_proba(new_X_test)[:, 1]
print('ROC_AUC score in test set: {}'.format(metrics.roc_auc_score(y_test, y_pred_prob)))

Accuracy score in training set: 0.5925925925925926
Precision score in training set: 0.5915492957746479
Recall score in training set: 0.9130434782608695
Confusion matrix:
[[ 6 29]
 [ 4 42]]
ROC_AUC score in training set: 0.6186335403726708
********************************************************************************
Accuracy score in test set: 0.5
Precision score in test set: 0.5416666666666666
Recall score in test set: 0.8666666666666667
Confusion matrix:
[[ 0 11]
 [ 2 13]]
ROC_AUC score in test set: 0.2909090909090909


In [414]:
predictions = {
    'prediction_with_sentiments': output_with_sentiment,
    'predictions_without_sentiments': output_without_sentiment,
    'date': list(train['date']) + list(test['date'])
}

In [415]:
pd.DataFrame(predictions).to_csv('logistic_predictions.csv', index=False)